# ID Classifier

Classify input image into UAE Natinoal ID, Passport, Driving License, Vehicle Registration and Other.

## Installing required packages

In [ ]:
pip install -Uq torch torchvision numpy matplotlib

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
facenet-pytorch 2.6.0 requires numpy<2.0.0,>=1.24.0, but you have numpy 2.2.6 which is incompatible.
facenet-pytorch 2.6.0 requires Pillow<10.3.0,>=10.2.0, but you have pillow 9.5.0 which is incompatible.
facenet-pytorch 2.6.0 requires torch<2.3.0,>=2.2.0, but you have torch 2.7.0 which is incompatible.
facenet-pytorch 2.6.0 requires torchvision<0.18.0,>=0.17.0, but you have torchvision 0.22.0 which is incompatible.
tensorflow 2.17.0 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 2.2.6 which is incompatible.


## Importing required packages

In [9]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

## Loading Data

In [8]:
!pip install scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 5.7 MB/s eta 0:00:00a 0:00:01


In [10]:
# !unzip img.zip

In [11]:
# Setup device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Data augmentation and normalization for training
# Just normalization for validation
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root='/content/img', transform=train_transforms)
val_dataset = datasets.ImageFolder(root='/content/img', transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=2)

Using device: cpu


FileNotFoundError: [Errno 2] No such file or directory: '/content/img'

## Training Model

In [ ]:
# Model setup
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

def train_model(num_epochs, patience):
    best_val_loss = np.inf
    epochs_no_improve = 0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        train_loss = running_loss / len(train_loader.dataset)

        # Validation phase
        model.eval()
        val_running_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * inputs.size(0)

        val_loss = val_running_loss / len(val_loader.dataset)

        print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

        # Early stopping logic
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            print("Saved Best Model")
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f'Early stopping triggered after {patience} epochs!')
                break
# Train the model
train_model(num_epochs=25, patience=5)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 152MB/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lea

Epoch 1/25 - Train Loss: 1.7438, Val Loss: 1.4068
Saved Best Model
Epoch 2/25 - Train Loss: 1.6718, Val Loss: 1.1461
Saved Best Model
Epoch 3/25 - Train Loss: 1.7697, Val Loss: 1.0843
Saved Best Model
Epoch 4/25 - Train Loss: 1.2394, Val Loss: 0.9527
Saved Best Model
Epoch 5/25 - Train Loss: 1.4670, Val Loss: 1.0283
Epoch 6/25 - Train Loss: 1.3015, Val Loss: 0.9697
Epoch 7/25 - Train Loss: 1.3270, Val Loss: 0.8214
Saved Best Model
Epoch 8/25 - Train Loss: 1.1903, Val Loss: 0.6729
Saved Best Model
Epoch 9/25 - Train Loss: 1.0376, Val Loss: 0.5852
Saved Best Model
Epoch 10/25 - Train Loss: 1.2399, Val Loss: 0.6636
Epoch 11/25 - Train Loss: 1.0733, Val Loss: 0.6179
Epoch 12/25 - Train Loss: 1.3845, Val Loss: 0.7926
Epoch 13/25 - Train Loss: 1.0264, Val Loss: 0.5994
Epoch 14/25 - Train Loss: 1.0604, Val Loss: 0.5067
Saved Best Model
Epoch 15/25 - Train Loss: 1.0703, Val Loss: 0.7157
Epoch 16/25 - Train Loss: 1.1747, Val Loss: 0.4532
Saved Best Model
Epoch 17/25 - Train Loss: 0.9384, Val Lo

## Evaluation the Model

In [5]:
from sklearn.metrics import classification_report

ModuleNotFoundError: No module named 'sklearn'

In [27]:
# Load the best model for further use
model.load_state_dict(torch.load('best_model.pth'))
model = model.to(device)

In [17]:
# Assuming you have a DataLoader for the test dataset named 'test_loader'
test_loader = DataLoader(datasets.ImageFolder(root='/content/img', transform=val_transforms), batch_size=32, shuffle=False, num_workers=4)

def evaluate_model(model, data_loader):
    model.eval()
    true_labels = []
    predictions = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.view(-1).tolist())
            true_labels.extend(labels.view(-1).tolist())

    print(classification_report(true_labels, predictions, target_names=data_loader.dataset.classes))
    print("Confusion Matrix:")
    print(confusion_matrix(true_labels, predictions))

# Evaluate the model
evaluate_model(model, test_loader)

NameError: name 'classification_report' is not defined

## Predicting on Prod

In [36]:
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchvision import models
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Class names based on your training
classes = ['id_card', 'licence_id', 'pasport', 'vhecial_licence']
#  ['UAE National ID Card', 'Passport', 'Driving License', 'Car Registration Card / Mulkiya', 'Other']

def load_model(model_path, num_classes, device):
    try:
        model = models.resnet50(weights=False)
        num_ftrs = model.fc.in_features
        model.fc = torch.nn.Linear(num_ftrs, num_classes)
        model.load_state_dict(torch.load(model_path, map_location=device))
        model = model.to(device)
        model.eval()
        logging.info("Model loaded successfully.")
        return model
    except Exception as e:
        logging.error(f"Failed to load model: {e}")
        raise

def prepare_image(image_path, input_size=224):
    try:
        transform = transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0)  # Adds a batch dimension
        logging.info("Image prepared successfully.")
        return image
    except Exception as e:
        logging.error(f"Failed to prepare image: {e}")
        raise

def classify_image(model, image, device):
    try:
        image = image.to(device)
        with torch.no_grad():
            outputs = model(image)
            _, predicted = torch.max(outputs, 1)
        predicted_class = classes[predicted.item()]
        logging.info(f"Image classified as {predicted_class}.")
        return predicted_class
    except Exception as e:
        logging.error(f"Failed to classify image: {e}")
        raise

def main(model_path, image_path, num_classes):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = load_model(model_path, num_classes, device)
    image = prepare_image(image_path)
    predicted_class = classify_image(model, image, device)
    print(f'Predicted Class: {predicted_class}')

In [41]:
%%timeit
model_path = '/content/best_model.pth'  # Path to your saved model
image_path = '/content/img/pasport/images (1).jpeg'  # Path to the image you want to classify
num_classes = len(classes)  # Total number of classes
main(model_path, image_path, num_classes)


Predicted Class: pasport
Predicted Class: pasport
Predicted Class: pasport
Predicted Class: pasport
Predicted Class: pasport
Predicted Class: pasport
Predicted Class: pasport
Predicted Class: pasport
589 ms ± 59.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Conclusion

Train on many images and evaluate with proper eval & test dataset